In [8]:
import time
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

file_path = 'data/data1_padi.csv'
df = pd.read_csv(file_path)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[["Luas Panen", "Curah hujan", "Kelembapan", "Suhu rata-rata"]].values)
y = df['Produksi'].values
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

Aktifasi Swish

In [9]:
# === MODEL ===
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(4, 50)
        self.fc2 = nn.Linear(50, 50)
        self.fc3 = nn.Linear(50, 1)

    def swish(self, x):
        return x * torch.sigmoid(x)

    def forward(self, x):
        x = self.swish(self.fc1(x))
        x = self.swish(self.fc2(x))
        x = self.fc3(x)
        return x

# === DEFINISI 5 FUNGSI LOSS ===
loss_functions = {
    "MSELoss": nn.MSELoss(),
    "L1Loss": nn.L1Loss(),
    "SmoothL1Loss": nn.SmoothL1Loss(),
    "HuberLoss": nn.HuberLoss(),
    "SumMSELoss": nn.MSELoss(reduction='sum')
}

# === TRAINING DAN EVALUASI ===
epochs = 1000
results = {}

for name, criterion in loss_functions.items():
    model = MLP()
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    start_time = time.time()
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        y_pred = model(X_train_tensor)
        loss = criterion(y_pred, y_train_tensor)
        loss.backward()
        optimizer.step()
    end_time = time.time()

    # Evaluasi
    model.eval()
    y_pred_test = model(X_test_tensor).detach().numpy()
    mse = mean_squared_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)
    exec_time = end_time - start_time

    results[name] = {
        "Execution Time (s)": exec_time,
        "MSE": mse,
        "R2 Score": r2
    }

# === OUTPUT HASIL ===
print("Hasil Evaluasi Model:")
for name, res in results.items():
    print(f"\n{name}:")
    print(f"  Execution Time: {res['Execution Time (s)']:.4f} s")
    print(f"  MSE: {res['MSE']:.4f}")
    print(f"  R² Score: {res['R2 Score']:.4f}")


Hasil Evaluasi Model:

MSELoss:
  Execution Time: 2.0282 s
  MSE: 92563860021.7988
  R² Score: 0.8953

L1Loss:
  Execution Time: 1.9248 s
  MSE: 91264487631.0854
  R² Score: 0.8968

SmoothL1Loss:
  Execution Time: 1.9576 s
  MSE: 96783030320.6991
  R² Score: 0.8905

HuberLoss:
  Execution Time: 1.9025 s
  MSE: 104528939383.9362
  R² Score: 0.8818

SumMSELoss:
  Execution Time: 1.6614 s
  MSE: 91699798613.5535
  R² Score: 0.8963
